# Projeto 1 - Ciência dos Dados

Nome: Arthur Gomes Chieppe

Nome: João Pedro Di Jardim Marquez

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from emoji import UNICODE_EMOJI

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'PS5.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo PS5.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.tail(5)

,Treinamento,Relevância:,Treinamento Limpo
295,discord https://t.co/q6oxummoxf\nsiga nossas m...,0,discord https//t.co/q6oxummoxfsiga nossas midi...
296,se tivesse a ps5 despedia me da marta e dos me...,0,se tivesse a ps5 despedia me da marta e dos me...
297,"desculpa pai, mas ps5 vai ter que vir no final...",0,desculpa pai mas ps5 vai ter que vir no final ...
298,"eu nem tô assim, quem tá são vocês\n\n#ps5show...",0,eu nem tô assim quem tá são vocês#ps5showcase ...
299,as vezes dá medo de pensar que o #ps5 chegue n...,1,as vezes dá medo de pensar que o #ps5 chegue n...


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)


,Teste,Relevância:
0,"sony nega, mas ações continuam em queda após r...",1.0
1,hoje tem \n\n#ps5,0.0
2,@prodigyrr não faça isso. ps5 ta uma completa ...,0.0
3,"hoje lançam o preço do ps5, tô só por saber qu...",1.0
4,"@1playergirl bom, nesse momento me sinto num h...",0.0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto trata-se do console de videogames PlayStation5. Quando extraímos os Tweets, havia uma espera coletiva acerca da livestream de lançamento do novo PlayStation 5. Assim , consideramos relevante especulações acerca do preço do novo console, bem como opiniões dos usuários sobre seu design. Ademais, consideramos algumas comparações entre o novo PlayStation e Xbox como relevantes, porém somente aquelas bem fundamentadas e intencionadas, já que consideramos irrelevantes comparações vagas e com discurso de ódio. Anúncios feitos acerca da live também foram consideradas como relevantes, com exceção de anúncios que tinham como objetivo somente a autopromoção dos usuários.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
#Limpando caracteres:
def limpa_tweets(dataframe):
    train_limpo = dataframe.iloc[:, 0]
    ls= []
    for w in train_limpo:
        w = w.replace(":",'')
        w = w.replace('"','')
        w = w.replace("'",'')
        w = w.replace("(",'')
        w = w.replace(")",'')
        w = w.replace('!','')
        w = w.replace('...','')
        w = w.replace(',','')
        w = w.replace('\n','')
        w = w.replace('/','')
        w = ''.join(' ' + char+' ' if char in UNICODE_EMOJI else char for char in w)
        w = w.replace('  ',' ')
        w = w.strip()
        w = w.casefold()
        ls.append(w)
    dataframe['Treinamento Limpo'] = ls
    return dataframe
train = limpa_tweets(train)


In [6]:
#train.to_excel(r'PS5Emoji.xlsx', sheet_name='Treinamento', index = False)

## Classificador:
Precisamos de:
P(R|T) = P(T|R) * P(R)/ P(T)




In [7]:
#Calculando Probabilidades:
def treinamento(dataframe):
    Rele = pd.Series(dtype=object)
    Irrele = pd.Series(dtype=object)

    for s in dataframe['Treinamento Limpo']:
        lsWords = pd.Series(s.split())
        for i in range(len(lsWords)):
            if dataframe['Relevância:'][i+1] == 1:
                Rele = Rele.append(lsWords)
            else:
                Irrele = Irrele.append(lsWords)
    return [Rele, Irrele]
    



In [8]:
Rele = treinamento(train)[0]
Irrele = treinamento(train)[1]
total_portugues = 100000 #Consideramos todas as palavras do portugues comumente usadas no twitter para maximizar smoothing.
#Calcaulando probabilidade em tweets relevantes:
lsRele = []
for v in Rele.value_counts():
    lsRele.append((v+1)/(total_portugues+Rele.count()))
lsKeys = Rele.value_counts().index.tolist()
dic_rele = {lsKeys[i]: lsRele[i] for i in range(len(lsKeys))} 
print(dic_rele)






{'o': 0.016312717945019907, 'e': 0.013191307438331539, 'ps5': 0.011898445990841802, 'de': 0.011589395445625131, 'que': 0.01130609911250985, 'do': 0.011177328052002906, 'a': 0.010693148864496788, 'um': 0.007242084442910638, 'é': 0.00647460892228924, 'no': 0.00623767017095646, 'dia': 0.006170709219492848, 'na': 0.0047336241842353315, 'bom': 0.004610003966148663, 'com': 0.004337009317873938, 'vai': 0.0039764503484544895, 'da': 0.003950696136353101, 'hoje': 0.003878584342469211, 'não': 0.0037961708637447656, 'pra': 0.003682852330498653, 'eu': 0.003626193063875597, 'mais': 0.003497422003368651, 'evento': 0.0030389970279639236, 'sony': 0.0030183936582828122, 'uma': 0.002961734391659756, 'preço': 0.0026269296343416964, '#ps5': 0.0025342144707766953, 'tem': 0.002359085828487249, 'para': 0.0023539349860669714, 'me': 0.0022921248770236373, 'em': 0.0022560689800816924, 'se': 0.002111845392313913, 'q': 0.002008828543908356, 'xbox': 0.0018749066409811325, 'os': 0.001859454113720299, 'por': 0.001838

In [9]:
#Calculando Probabilidade em tweets irrelevantes:
lsIrrele = []
for v in Irrele.value_counts():
    lsIrrele.append((v+1)/(total_portugues+Irrele.count()))
lsKeys = Irrele.value_counts().index.tolist()
dic_irrele = {lsKeys[i]: lsIrrele[i] for i in range(len(lsKeys))}
print(dic_irrele)

{'o': 0.018176559171380297, 'e': 0.015040769852347022, 'de': 0.013167560420186587, 'que': 0.012575295673253507, 'do': 0.012309006097113055, 'ps5': 0.012272276500404026, 'a': 0.011992213325497686, 'um': 0.00810805847351796, 'é': 0.007162271358260486, 'no': 0.00705667376772203, 'dia': 0.007033717769778887, 'na': 0.005371703518695365, 'bom': 0.0052615147285682805, 'com': 0.004917174759421141, 'da': 0.004453463600969662, 'vai': 0.0043983692059061195, 'hoje': 0.004219312421949607, 'não': 0.004191765224417836, 'pra': 0.004072394035113495, 'eu': 0.0040172996400499525, 'mais': 0.0038749724528024682, 'sony': 0.003333210901344303, 'evento': 0.0033286197017556745, 'uma': 0.003319437302578418, 'preço': 0.0028373613457724233, '#ps5': 0.0027684933519429955, 'para': 0.0026583045618159113, 'me': 0.0025802541688092263, 'tem': 0.0025435245721001985, 'em': 0.00253893337251157, 'se': 0.0023323293910232864, 'q': 0.002194593403364431, 'os': 0.002130316609123632, 'por': 0.0020981782120032323, 'isso': 0.00207

In [10]:
print(sum(lsIrrele))
print(sum(lsRele))

0.5509072210387196
0.4961651978181018


In [11]:
def classifica(dataframe, dic_rele, dic_irrele):
    lsClassificacao = []
    for t in dataframe['Treinamento Limpo']:
        lsWord = t.split()
        lsRele = []
        lsIrrele = []
        for w in lsWord:
            if w in dic_rele:
                lsRele.append(dic_rele[w])
            if w in dic_irrele:
                lsIrrele.append(dic_irrele[w])
            else:
                lsRele.append(1/(total_portugues+Rele.count()))
                lsIrrele.append(1/(total_portugues+Irrele.count()))
        if np.prod(lsRele) >= np.prod(lsIrrele):
            lsClassificacao.append(1)
        else:
            lsClassificacao.append(0)
    return lsClassificacao
teste = pd.Series(classifica(train, dic_rele, dic_irrele))
teste.value_counts()

0    239
1     61
dtype: int64

In [12]:
#Comparando com as entradas manualmente digitadas:
train['Relevância:'].value_counts()

0    151
1    149
Name: Relevância:, dtype: int64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**